## Example of usage of the individual modules ##

This jupyter notebook shows how to use the different modules contained in CABARET independently for different computations.

First, we have to import all the modules needed

In [1]:
import numpy as np
import sys
import time
import os

# To run locally and find the modules in /src
cabaret_src_folder = '/Users/anabel/Documents/PhD/Code/pyCabaret/src' # Change to your folder path
sys.path.insert(0, cabaret_src_folder)

mutation_folder = "/Users/anabel/Documents/PhD/Code/Mpp_test/Mutationpp/" # Your folder
my_distribution = "macosx-10.15-x86_64-3.9" # Your particular distribution
sys.path.append(mutation_folder + "_skbuild/" + my_distribution +"/cmake-install/interface/python/mutationpp")

import _mutationpp as mpp
from reservoir import reservoir
from massflow import massflow
from shock import shock
from total import total
from heatflux import heatflux
import rebuilding_setup as setup
import reading_input as input_data

In [2]:
# Free stream conditions
T_1 = 3500.0
p_1 = 5000.0
M_1 = 2.0

In [3]:
# Mutation++ mixture setup
mix = setup.setup_mpp()

## Reservoir ##

To compute the reservoir quantities, namely the pressure and temperature, we solve the following system of equations

$h_{1}(T_{1},P_{1}) + \dfrac{1}{2}v_{1}^{2} = h_{\mathrm{0}}(T_{0},P_{0})$

$s_{1}(T_{1},P_{1}) = s_{\mathrm{0}}(T_{0},P_{0})$,

where $h$ and $s$ are the mixture enthalpy and entropy, respectively.

The equations are not solved on $T_{0}, P_{0}$ but on the proportionality variables $c_{1}, c_{2}$ such that $T_{0} = c_{1}\times T_{1}$ and $P_{0} = c_{2}\times P_{1}$. This simplifies the optimization algorithm by handling quantities of similar orders of magnitude.

In the following "options" python dictionary, you have to specify the initial conditions for both $c_{1}$ in the "temperature" field, and $c_{2}$ in the "pressure" field.

In [ ]:
# Specify module options
options = {"pressure": 10000.0, 
           "temperature": 100.0,
           "robust": "Yes"}

In [ ]:
setup.mixture_states(mix)["reservoir"].equilibrate(T_1,p_1)
v_1 = M_1*setup.mixture_states(mix)["reservoir"].equilibriumSoundSpeed()
h_1 = setup.mixture_states(mix)["reservoir"].mixtureHMass() + (0.5*v_1**2)
s_1 = setup.mixture_states(mix)["reservoir"].mixtureSMass()

start_time = time.time()
T0,p0,v0 = reservoir(T_1,p_1,h_1,s_1,1.0e-06,mix,"reservoir",options)
end_time = time.time()

exec_time = end_time - start_time

print('T0 = '+ "{:.2f}".format(T0)+' K;', 'p0 = '+ "{:.2f}".format(p0)+' Pa;', 'v0 = '+ "{:.2f}".format(v0)+' m/s')
print('Execution time = '+"{:.4f}".format(exec_time), ' seconds = '+"{:.4f}".format(exec_time/60), ' minutes')

## Mass flow computation ##

The mass flow is computed by using the quantities that define the throat of the nozzle. Any wind tunnel that expands to supersonic/hypersonic speeds will have a throat. This allows us to know a velocity of Mach = 1 is reached there, from which we can deduce the mass flow along the stagnation line of the nozzle. The equations solved are

$h_{1}(T_{1},P_{1}) + \dfrac{1}{2}v_{1}^{2} = h_{\mathrm{throat}}(T_{\mathrm{throat}},P_{\mathrm{throat}}) + \dfrac{1}{2}a_{\mathrm{throat}}(T_{\mathrm{throat}},P_{\mathrm{throat}})^{2}$

$s_{1}(T_{1},P_{1}) = s_{\mathrm{throat}}(T_{\mathrm{throat}},P_{\mathrm{throat}})$,

where $a_{\mathrm{throat}}$ is the local equilibrium speed of sound. Once we solve for $T_{\mathrm{throat}}$ and $P_{\mathrm{throat}}$ we compute the mass flow as

$\dot{m} = \rho_{\mathrm{throat}}(T_{\mathrm{throat}},P_{\mathrm{throat}})\cdot A_{\mathrm{throat}}\cdot a_{\mathrm{throat}}(T_{\mathrm{throat}},P_{\mathrm{throat}})$,

with $A_{\mathrm{throat}}$ being the throat area.

The variables used to solve the above equations are the same as for the reservoir problem.

In [ ]:
# Specify module options
options = {"pressure": 10.0, 
           "temperature": 2.0,
           "robust": "No"}

A_t = 9.621e-04

In [ ]:
start_time = time.time()
mfl = massflow(T_1,p_1,h_1,s_1,A_t,1.0e-06,mix,"throat",options)
end_time = time.time()
exec_time = end_time - start_time

print('mass flow = ', str(mfl*1000.0)+' g/s')
print('Execution time = '+"{:.4f}".format(exec_time), ' seconds = '+"{:.4f}".format(exec_time/60), ' minutes')

## Shocking ##

We solve the following system of equations known as the Rankine-Hugoniot relations

$\rho_{1}v_{1} = \rho_{2}v_{2}$,

$P_{1} + \rho_{1}v_{1}^{2} = P_{2} + \rho_{2}v_{2}^{2}$,

$(\rho_{1}E_{1} + P_{1})v_{1} = (\rho_{2}E_{2} + P_{2})v_{2}$.

The variable names are quite self-explanatory.

In practice, we solve for the density ratio $\rho_{1}/\rho_{2}$, while updating the temperature to match the total energy and the corresponding pressure.

In [ ]:
# Specify module options
options = {"ratio": 0.2, 
           "robust": "No"}

In [ ]:
preshock_state = [T_1,p_1,M_1]

start_time = time.time()
T2,p2,v2 = shock(preshock_state,mix,options)
end_time = time.time()
exec_time = end_time - start_time

print('T2 = '+ "{:.2f}".format(T2)+' K;', 'p2 = '+ "{:.2f}".format(p2)+' Pa;', 'v2 = '+ "{:.2f}".format(v2)+' m/s')
print('Execution time = '+"{:.4f}".format(exec_time), ' seconds = '+"{:.4f}".format(exec_time/60), ' minutes')

## Total quantities ##

Computing the total or stagnation quantities of a given state (temperature, pressure and velocity) is equivalent to doing a reservoir computation. Isentropic relations to deccelerate the flow to stagnant conditions. We solve the same equations.

Now, the variables we solve for are proportional to the state from which we want to compute the total quantities, in this case $T_{2}, P_{2}, v_{2}$. This is why we choose to initialize these variables with 1.0.

In [ ]:
options = {"pressure": 1.0, 
           "temperature": 1.0,
           "robust": "No"}

In [ ]:
start_time = time.time()
Tt2,pt2,vt2 = total(T2,p2,v2,1.0e-06,mix,"total",options)
end_time = time.time()
exec_time = end_time - start_time

print('Tt2 = '+ "{:.2f}".format(Tt2)+' K;', 'pt2 = '+ "{:.2f}".format(pt2)+' Pa;', 'vt2 = '+ "{:.2f}".format(vt2)+' m/s')
print('Execution time = '+"{:.4f}".format(exec_time), ' seconds = '+"{:.4f}".format(exec_time/60), ' minutes')

## Heat flux ##

We solve the Fay-Riddell equation for a fully catalytic wall in equilibrium. For a Lewis number of 1, the equation is

$Q_{\mathrm{w}} = 0.763\cdot \mathrm{Pr}^{-0.6}\cdot(\rho_{t2} \cdot \mu_{t2})^{0.4}\cdot(\rho_{\mathrm{w}} \cdot \mu_{\mathrm{w}})^{0.1}\cdot \sqrt(\beta)\cdot(h_{t2} -h_{\mathrm{w}})$,

where the subscript $\mathrm{w}$ stands for the wall state. The dynamic viscosities $\mu_{t2}$ and $\mu_{\mathrm{w}}$ are functions of the respective states, such as $\mu_{t2}(T_{t2},P_{t2})$ and $\mu_{\mathrm{w}}(T_{\mathrm{w}},P_{\mathrm{w}})$ with $P_{\mathrm{w}}$ taken as $P_{t2}$.

The velocity gradient $\beta$ is modeled through the Modified Newtonian Theory as $\beta = \dfrac{1}{R_{\mathrm{eff}}}\sqrt{\dfrac{2\cdot(P_{t2} - P_{1})}{\rho_{t2}}}$.

In [ ]:
pr = 0.713 # Prandtl number
L = 1.0 # Lewis number
reff = 0.025 # Effective radius
T_w = 350.0 # Wall temperature

mix.equilibrate(Tt2, pt2)
ht2 = mix.mixtureHMass()

In [ ]:
start_time = time.time()
qw = heatflux(mix, pr, L, p_1, pt2, Tt2, ht2, reff, T_w)
end_time = time.time()

exec_time = end_time - start_time

print('Qw = '+ "{:.2f}".format(qw)+' W/m2')
print('Execution time = '+"{:.4f}".format(exec_time), ' seconds = '+"{:.4f}".format(exec_time/60), ' minutes')